In [1]:
#Imports (FIXED)

import os

from langchain_google_genai import (
    GoogleGenerativeAIEmbeddings,
    ChatGoogleGenerativeAI
)

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

from langchain_core.messages import SystemMessage, HumanMessage


C:\Users\ARUL KUMARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
#  Google API Key

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = "YOUR API KEY"


In [3]:
# LLM (FIXED – init_chat_model REMOVED)
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    max_new_tokens=200
)

llm = HuggingFacePipeline(pipeline=pipe)



C:\Users\ARUL KUMARAN\AppData\Local\Temp\ipykernel_29152\2598141822.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [4]:
# Load Web Documents
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(
    "PYTHON PROGRAMMING NOTES.pdf"
)

raw_docs = loader.load()



In [5]:
# Text Splitting
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150
)

docs = splitter.split_documents(raw_docs)


In [6]:
from langchain_community.embeddings import FakeEmbeddings


embeddings = FakeEmbeddings(size=384)
print("Fake embeddings loaded ✅")



Fake embeddings loaded ✅


In [7]:
#Vector Store + Retriever

vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory="./chroma_db",
    collection_name="selfrag-demo"
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}

)


In [8]:
#Self-RAG Function (FIXED)

def self_rag(question: str):

    # Step 0: Retrieve context
    context_docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in context_docs)

    # Step 1: Initial Answer
    prompt = f"""
Answer the question using ONLY the context below.

Context:
{context}

Question:
{question}
"""
    answer = llm.invoke(prompt)
    print("Initial Answer:\n", answer)

    # Step 2: Critique
    critique_prompt = f"""
You are a fact-checker.

Context:
{context}

Answer:
{answer}

Is the answer fully supported by the context?
Say yes or no and explain.
"""
    critique = llm.invoke(critique_prompt)
    print("\nCritique:\n", critique)

    # Step 3: Revise if needed
    if "no" in critique.lower():
        revise_prompt = f"""
Improve the answer so it strictly matches the context.

Context:
{context}

Original Answer:
{answer}

Revised Answer:
"""
        revised = llm.invoke(revise_prompt)
        return revised

    return answer


In [9]:
while True:
    query = input("\nAsk your question (type 'exit' to quit): ")

    if query.lower() == "exit":
        break

    response = self_rag(query)   # ✅ Direct call
    print("\n🔍 Final Answer:\n", response)



Ask your question (type 'exit' to quit):  what is python?


Token indices sequence length is longer than the specified maximum sequence length for this model (614 > 512). Running this sequence through the model will result in indexing errors


Initial Answer:
 python

Critique:
 no

🔍 Final Answer:
 python



Ask your question (type 'exit' to quit):  what is function?


Initial Answer:
 Output: Function definitions do not alter the flow of execution of the program, but remember that statements inside the function are not executed until the function is called.

Critique:
 Output: yes

🔍 Final Answer:
 Output: Function definitions do not alter the flow of execution of the program, but remember that statements inside the function are not executed until the function is called.


...


Ask your question (type 'exit' to quit):  what is keywords?


Initial Answer:
 tuple

Critique:
 yes

🔍 Final Answer:
 tuple



Ask your question (type 'exit' to quit):  what is keyword arguments?


Initial Answer:
 a keyword argument

Critique:
 no

🔍 Final Answer:
 a keyword argument



Ask your question (type 'exit' to quit):  exit
